# Worksheet 6 example code

Calculate the baseline (1961-1990) and future (2021-2050) 90th percentile of maximum temperature and the difference between them. Do this for cahpa and cahpb.

In [ ]:
# Code preamble - these libraries will be used in this worksheet.
# This code block needs to be re-run every time you restart this worksheet!
%matplotlib inline 
import os
import iris
import iris.coord_categorisation
import iris.quickplot as qplt
import iris.plot as iplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import dask
dask.config.set(get=dask.get)
import dask.array as da
from iris.analysis import Aggregator

# /data/users/ssadri/pyprecis/monthly
# Some helpful data locations
DATADIR = '/project/ciid/projects/PRECIS/worksheets/data'
APHRODIR = os.path.join(DATADIR, 'APHRODITE')
CRUDIR = os.path.join(DATADIR, 'CRU')
CLIMDIR = os.path.join(DATADIR, 'climatology')
MODELDIR = os.path.join(DATADIR, 'netcdf')
JOBIDS = ['cahpa', 'cahpb']
TIME_PERIODS = {'baseline':'1981_1983', 'future':'2021_2050'}

In [ ]:
for runid in JOBIDS:
    for t in TIME_PERIODS.keys():
        
    data_path = 'daily/' + runid + '/03236.max/'
    for tp, tpn in zip(*(time_periods, time_period_names)):
        filename = runid + 'a.pa.' + tp + '.03236.max.rr8.degC.nc'
        model_precip = iris.load_cube(data_path + filename)
        model_pc90 = model_precip.collapsed('time', iris.analysis.PERCENTILE, percent=90.0)
        model_file_out = runid + 'a.pc90.03236.max.' + tpn + '.nc'
        iris.save(model_pc90, path_clim + model_file_out)

Calculate the frequency of warm days in the future (extreme index TX90P), i.e. the number of days which exceed the 90th percentile temperatures in the baseline.  Then calculate the numbers of days as a percentage.

In [ ]:
runids = ['cahpa', 'cahpb']
path_clim = 'daily/climatology/'

for runid in runids:
    data_path = 'daily/' + runid + '/03236.max/'
    file_fut = job + 'a.pa.2150.03236.max.rr8.degC.nc'
    tx = iris.load_cube(data_path + file_fut)

    file_pc90 = runid + 'a.pc90.03236.max.baseline.nc'
    pc90 = iris.load_cube(path_clim + file_pc90)

    dims = tx.shape[1:]
    nwarmdays = pc90.copy()
    nwarmdays.data = np.zeros(dims)

# Use broadcasting to identify all cells where daily temperatures in the future exceed the 90th percentile
    tx_gt_pc90 = np.where(tx.data > pc90.data, 1, 0)
    nwarmdays.data = np.sum(tx_gt_pc90, axis=0)

    print "Saving numbers of warm days in the future from " + job
    iris.save(nwarmdays, path_clim + job + 'a.TX90P.nc')

Plot the numbers of warm days in the future and the percentage of warm days.

In [ ]:
ndays = [10800.0, 10957.0]
runids = ['cahpa', 'cahpb']
path_clim = 'daily/climatology/'

plt.figure(figsize=(12, 12))

for i, job in enumerate(runids):
    nwarmdays = iris.load_cube(path_clim + job + 'a.TX90P.nc')
    nwd_pcent = iris.analysis.maths.divide(iris.analysis.maths.multiply(nwarmdays, 100), ndays[i])
    iris.save(nwd_pcent, job + 'a.TX90P.per.nc')

    plotnum = 1 + 2*i
    plt.subplot(2, 2, plotnum)
    qplt.pcolormesh(nwarmdays, vmin=0, vmax=10000)
    plt.title(job + ': Number of warm days')
    plt.subplot(2, 2, plotnum+1)
    qplt.pcolormesh(nwd_pcent, vmin=0, vmax=100)
    plt.title(job + ': Percentage of warm days')

plt.tight_layout()
plt.show()

Calculate the percentage of total precipitation which falls on very wet days in the future
(where a very wet day is one on which daily rainfall exceeds the 95th percentile of the baseline)
over Malaysia.

In [ ]:
runids = ['cahpa', 'cahpb']
ndays = [10800.0, 10957.0]
path_clim = 'daily/climatology/'

# Coordinates of a box around Malaysia
malay_lons = np.array([99.0, 99.0, 122.0, 122.0])
malay_lats = np.array([ 0.0,  9.0,   0.0,   9.0])

# Load a cube on the rotated grid
rotg = iris.load_cube(path_clim + 'cahpaa.wetday.pcent.baseline.nc')
rcs = rotg.coord('grid_latitude').coord_system

# Get the rotated pole coordinates
pole_lat = rcs.grid_north_pole_latitude
pole_lon = rcs.grid_north_pole_longitude

# Convert the coordinates of a box around Malaysia from real coordinates to rotated polar coordinates
grid_lons, grid_lats = iris.analysis.cartography.rotate_pole(malay_lons, malay_lats, pole_lon, pole_lat)

# Find the max / min of the lons / lats on the rotated grid.  They will be used to extract the data around Malaysia
# N.B. The conversion to float is needed, as numpy data are of type float64 by default. If the coordinate limits
# are passed as float64, they are interpreted as a list of two floats and the program will stop with an error:
# ValueError: setting an array element with a sequence.
lon_0 = float(min(grid_lons))
lon_1 = float(max(grid_lons))
lat_0 = float(min(grid_lats))
lat_1 = float(max(grid_lats))

# Set up constraints on the rotated grid for Malaysia
lon_con = iris.Constraint(grid_longitude = lambda x: lon_0 <= x <= lon_1)
lat_con = iris.Constraint(grid_latitude = lambda cell: lat_0 <= cell <= lat_1)

for i, job in enumerate(runids):
    data_path = 'daily/' + job + '/05216/'
    file_f = job + 'a.pa.2150.05216.rr8.mmday.nc'
#   precip = iris.load_cube(data_path + file_f, lon_con).intersection(grid_latitude = (-14.767, -5,623))
    precip = iris.load_cube(data_path + file_f, lat_con & lon_con)
    precip_pc95 = iris.load_cube(path_clim + 'cahpaa.pc95.05216.future.mmday.nc', lat_con & lon_con)
    
# Use broadcasting to identify all cells in precip where p95 is exceeded
    pre_gt_pc95 = np.where(precip.data > precip_pc95.data, precip.data, 0.0)
    pre_p95 = np.sum(pre_gt_pc95, axis=0)
    pre_tot = precip.collapsed('time', iris.analysis.SUM)
    pre_tot.data = np.divide(pre_p95, pre_tot.data, out=np.zeros_like(pre_tot.data), where = pre_tot.data != 0)
    pre_tot = iris.analysis.maths.multiply(pre_tot, 100)
    file_out = job + 'a.R95pTOT.future.nc'
    iris.save(pre_tot, path_clim + file_out)

Plot the percentages of heavy rainfall in the future.

In [ ]:
runids = ['cahpa', 'cahpb']
path_clim = 'daily/climatology/'

plt.figure(figsize=(12, 12))

for i, job in enumerate(runids):
    filename = job + 'a.R95pTOT.future.nc'
    pcent_heavy_rain = iris.load_cube(path_clim + filename)
    plotnum = 1 + i
    plt.subplot(2, 1, plotnum)
    qplt.pcolormesh(pcent_heavy_rain, vmin=0, vmax=100)
    plt.title(job + ': Percentage of heavy rain over Malaysia in the future')

plt.tight_layout()
plt.show()